In [1]:
'''

截取所有条件的epoch，然后进行清洗。

所有epoch ，然后再来考虑 evoked


ASSR 实验
204  230

oddball 实验
stdBA 110  odd  111
stdT1 130       131
stdT4 140       141
stdTA 120       121

ASSR刺激：2s的刺激，0.1的前后ramp，1.9的ITI
oddball刺激：总共1.5
'''


'\n\n截取所有条件的epoch，然后进行清洗。\n\n所有epoch ，然后再来考虑 evoked\n\n\n\nASSR 实验\n204  230\n\noddball 实验\nstdBA 110  odd  111\nstdT1 130       131\nstdT4 140       141\nstdTA 120       121\n\nASSR刺激：2s的刺激，0.1的前后ramp，1.9的ITI\noddball刺激：总共1.5\n'

In [1]:
import mne
import os.path as op
import mne
from mne.preprocessing import ICA
import matplotlib.pyplot as plt
from copy import deepcopy
import numpy as np
from autoreject import AutoReject

In [2]:
'''

{'110': 1, '111': 2, '120': 3, '121': 4, '130': 5, '131': 6, '140': 7, '141': 8, '204': 9, '230': 10, '50': 11, '51': 12}


4Hz:'204': 9,
30Hz:'230': 10

ba 200
ta 201
ji1 210
ji4 211

'''
sub_ids = [
            'pre4014_y'
          ]



#for sub_id,i in zip(sub_ids,range(len(sub_ids))):
    #读入之前处理好的reconst_raw

sub_id = sub_ids[0]
i = 0
save_path_ica = '.\data\\4raw_ica\\'
reconst_raw=mne.io.read_raw_fif(save_path_ica + sub_id + '.fif')
# 列出所有条件 对应的 mark数字。
# 只要oddball
target_events_oddball = {
                            '200': 1, '201': 2, '210': 3, '211': 4
                        }

#先找events，epoch和average都是一步的事情。
custom_mapping = target_events_oddball
(events_from_annot,
 event_dict) = mne.events_from_annotations(reconst_raw, event_id=custom_mapping)
#还不如直接对 events_from_annot 进行切片
#先遍历，把合适的单词筛序出来
#对应的vocabulary已经列出，然后怎么提取呢？
events_oddball = events_from_annot
count_remembered_events = len(events_oddball)
epochs_oddball = mne.Epochs(reconst_raw, events_oddball, baseline =(-0.5,0),tmin=-0.5, tmax=1, event_id=None,
                    preload=True, event_repeated='merge')

#查看所有mark dict
#events_1 = mne.events_from_annotations(reconst_raw)

Opening raw data file .\data\4raw_ica\pre4014_y.fif...


C:\Users\SilverBullet\AppData\Local\Temp\ipykernel_21784\1668997415.py:27: RuntimeWarning: This filename (.\data\4raw_ica\pre4014_y.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  reconst_raw=mne.io.read_raw_fif(save_path_ica + sub_id + '.fif')


    Range : 0 ... 5081399 =      0.000 ...  5081.399 secs
Ready.
Used Annotations descriptions: ['200', '201', '210', '211']
Not setting metadata
860 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 860 events and 1501 original time points ...
8 bad epochs dropped


In [29]:
reconst_raw.plot()

In [30]:
'''
所有oddball试次一起进行autoreject
'''
save_path_epoch = '.\data\\5epoch_oddball\\'
epochs_oddball.save(save_path_epoch + sub_id + '-epo.fif', overwrite=True)
    #用autoreject处理一下
n_interpolates = np.array([1, 4, 32])
consensus_percs = np.linspace(0, 0.8, 11)
picks = mne.pick_types(epochs_oddball.info, meg=False, eeg=True, stim=False, eog=False,
                       include=[], exclude=[])
ar_oddball = AutoReject(n_interpolates,consensus_percs,picks=picks,
                thresh_method='random_search',random_state=23)
ar_oddball.fit(epochs_oddball)
epochs_clean_oddball = ar_oddball.transform(epochs_oddball)


Running autoreject on ch_type=eeg


  0%|          | Creating augmented epochs : 0/62 [00:00<?,       ?it/s]

  0%|          | Computing thresholds ... : 0/62 [00:00<?,       ?it/s]

  0%|          | Repairing epochs : 0/852 [00:00<?,       ?it/s]

  0%|          | n_interp : 0/3 [00:00<?,       ?it/s]

...

  0%|          | Repairing epochs : 0/852 [00:00<?,       ?it/s]

  0%|          | Fold : 0/10 [00:00<?,       ?it/s]





Estimated consensus=0.48 and n_interpolate=1


  0%|          | Repairing epochs : 0/852 [00:00<?,       ?it/s]

No bad epochs were found for your data. Returning a copy of the data you wanted to clean. Interpolation may have been done.


In [31]:
'''
8种trial分开保存。
std_BA	110 1
odd_TA	111 2
std_ta	120 3
odd_ba	121 4

std_t1ji	130 5
odd_t4ji	131 6
std_t4ji	140 7
odd_t1ji	141 8
'''
epochs_clean_oddball_std_BA = epochs_clean_oddball['1']
epochs_clean_oddball_odd_TA = epochs_clean_oddball['2']
# epochs_clean_oddball_std_ta = epochs_clean_oddball['3']
# epochs_clean_oddball_odd_ba = epochs_clean_oddball['4']
epochs_clean_oddball_std_t1ji = epochs_clean_oddball['3']
epochs_clean_oddball_odd_t4ji= epochs_clean_oddball['4']
# epochs_clean_oddball_std_t4ji = epochs_clean_oddball['7']
# epochs_clean_oddball_odd_t1ji = epochs_clean_oddball['8']

evoked_original_oddball_std_BA = list()
evoked_clean_oddball_std_BA = list()
evoked_original_oddball_odd_TA = list()
evoked_clean_oddball_odd_TA = list()
evoked_original_oddball_std_ta = list()
evoked_clean_oddball_std_ta = list()
evoked_original_oddball_odd_ba = list()
evoked_clean_oddball_odd_ba = list()
evoked_original_oddball_std_t1ji = list()
evoked_clean_oddball_std_t1ji = list()
evoked_original_oddball_odd_t4ji = list()
evoked_clean_oddball_odd_t4ji = list()
evoked_original_oddball_std_t4ji = list()
evoked_clean_oddball_std_t4ji = list()
evoked_original_oddball_odd_t1ji = list()
evoked_clean_oddball_odd_t1ji = list()

In [32]:
# 清洗完之后就要分开了。
#先average看下结果，对比
#如果结果不错就可以分别保存下来。
evoked_original_oddball_std_BA.append(epochs_oddball['1'].average())#没有autoreject的
evoked_clean_oddball_std_BA.append(epochs_clean_oddball_std_BA.average())

save_path_epoch_clean = '.\data\\6epoch_clean_oddball\\'
epochs_clean_oddball_std_BA.save(save_path_epoch_clean + sub_id + '_std_BA-epo.fif', overwrite=True)
save_path_evoked = '.\data\\7evoked_oddball\\'
#为什么这里加i?因为是list，epochs不是list
evoked_clean_oddball_std_BA[i].save(save_path_evoked + sub_id + '_std_BA-ave.fif',overwrite=True)


#先average看下结果，对比
#如果结果不错就可以分别保存下来。
evoked_original_oddball_odd_TA.append(epochs_oddball['2'].average())#没有autoreject的
evoked_clean_oddball_odd_TA.append(epochs_clean_oddball_odd_TA.average())

save_path_epoch_clean = '.\data\\6epoch_clean_oddball\\'
epochs_clean_oddball_odd_TA.save(save_path_epoch_clean + sub_id + '_odd_TA-epo.fif', overwrite=True)
save_path_evoked = '.\data\\7evoked_oddball\\'
#为什么这里加i?因为是list，epochs不是list
evoked_clean_oddball_odd_TA[i].save(save_path_evoked + sub_id + '_odd_TA-ave.fif',overwrite=True)


# #先average看下结果，对比
# #如果结果不错就可以分别保存下来。
# evoked_original_oddball_std_ta.append(epochs_oddball['3'].average())#没有autoreject的
# evoked_clean_oddball_std_ta.append(epochs_clean_oddball_std_ta.average())
#
# save_path_epoch_clean = '.\data\\6epoch_clean_oddball\\'
# epochs_clean_oddball_std_ta.save(save_path_epoch_clean + sub_id + '_std_ta-epo.fif', overwrite=True)
# save_path_evoked = '.\data\\7evoked_oddball\\'
# #为什么这里加i?因为是list，epochs不是list
# evoked_clean_oddball_std_ta[i].save(save_path_evoked + sub_id + '_std_ta-ave.fif',overwrite=True)
#
#
#
# #先average看下结果，对比
# #如果结果不错就可以分别保存下来。
# evoked_original_oddball_odd_ba.append(epochs_oddball['4'].average())#没有autoreject的
# evoked_clean_oddball_odd_ba.append(epochs_clean_oddball_odd_ba.average())
#
# save_path_epoch_clean = '.\data\\6epoch_clean_oddball\\'
# epochs_clean_oddball_odd_ba.save(save_path_epoch_clean + sub_id + '_odd_ba-epo.fif', overwrite=True)
# save_path_evoked = '.\data\\7evoked_oddball\\'
# #为什么这里加i?因为是list，epochs不是list
# evoked_clean_oddball_odd_ba[i].save(save_path_evoked + sub_id + '_odd_ba-ave.fif',overwrite=True)



#先average看下结果，对比
#如果结果不错就可以分别保存下来。
evoked_original_oddball_std_t1ji.append(epochs_oddball['3'].average())#没有autoreject的
evoked_clean_oddball_std_t1ji.append(epochs_clean_oddball_std_t1ji.average())

save_path_epoch_clean = '.\data\\6epoch_clean_oddball\\'
epochs_clean_oddball_std_t1ji.save(save_path_epoch_clean + sub_id + '_std_t1ji-epo.fif', overwrite=True)
save_path_evoked = '.\data\\7evoked_oddball\\'
#为什么这里加i?因为是list，epochs不是list
evoked_clean_oddball_std_t1ji[i].save(save_path_evoked + sub_id + '_std_t1ji-ave.fif',overwrite=True)



#先average看下结果，对比
#如果结果不错就可以分别保存下来。
evoked_original_oddball_odd_t4ji.append(epochs_oddball['4'].average())#没有autoreject的
evoked_clean_oddball_odd_t4ji.append(epochs_clean_oddball_odd_t4ji.average())

save_path_epoch_clean = '.\data\\6epoch_clean_oddball\\'
epochs_clean_oddball_odd_t4ji.save(save_path_epoch_clean + sub_id + '_odd_t4ji-epo.fif', overwrite=True)
save_path_evoked = '.\data\\7evoked_oddball\\'
#为什么这里加i?因为是list，epochs不是list
evoked_clean_oddball_odd_t4ji[i].save(save_path_evoked + sub_id + '_odd_t4ji-ave.fif',overwrite=True)

# #先average看下结果，对比
# #如果结果不错就可以分别保存下来。
# evoked_original_oddball_std_t4ji.append(epochs_oddball['7'].average())#没有autoreject的
# evoked_clean_oddball_std_t4ji.append(epochs_clean_oddball_std_t4ji.average())
#
# save_path_epoch_clean = '.\data\\6epoch_clean_oddball\\'
# epochs_clean_oddball_std_t4ji.save(save_path_epoch_clean + sub_id + '_std_t4ji-epo.fif', overwrite=True)
# save_path_evoked = '.\data\\7evoked_oddball\\'
# #为什么这里加i?因为是list，epochs不是list
# evoked_clean_oddball_std_t4ji[i].save(save_path_evoked + sub_id + '_std_t4ji-ave.fif',overwrite=True)
#
# #先average看下结果，对比
# #如果结果不错就可以分别保存下来。
# evoked_original_oddball_odd_t1ji.append(epochs_oddball['8'].average())#没有autoreject的
# evoked_clean_oddball_odd_t1ji.append(epochs_clean_oddball_odd_t1ji.average())
#
# save_path_epoch_clean = '.\data\\6epoch_clean_oddball\\'
# epochs_clean_oddball_odd_t1ji.save(save_path_epoch_clean + sub_id + '_odd_t1ji-epo.fif', overwrite=True)
# save_path_evoked = '.\data\\7evoked_oddball\\'
# #为什么这里加i?因为是list，epochs不是list
# evoked_clean_oddball_odd_t1ji[i].save(save_path_evoked + sub_id + '_odd_t1ji-ave.fif',overwrite=True)

In [33]:
"""
查看效果
"""
%matplotlib qt

fig_original = evoked_original_oddball_std_t1ji[i].plot(window_title = sub_id)
fig_original.savefig(save_path_evoked+'_original_' + sub_id)
fig_clean = evoked_clean_oddball_odd_t4ji[i].plot(window_title = sub_id)
fig_clean.savefig(save_path_evoked+'_clean_'+ sub_id)

